In [73]:
from snowflake.snowpark import Session
from snowflake.snowpark.dataframe import col
import os
import json
import pytest
from query_extractor import extract_query
# Parse the JSON string into a Python dictionary
json_str = open('./config/config.json','r+').read()
config_dict = json.loads(json_str)

In [77]:
import sys

if 'pytest' in sys.modules:
    for i in sys.modules:
        print(i)

sys
builtins
_frozen_importlib
_imp
_thread
_warnings
_weakref
_io
marshal
nt
winreg
_frozen_importlib_external
time
zipimport
_codecs
codecs
encodings.aliases
encodings
encodings.utf_8
_signal
_abc
abc
io
__main__
_stat
stat
_collections_abc
genericpath
ntpath
os.path
os
_sitebuiltins
encodings.cp1252
__future__
itertools
keyword
_operator
operator
reprlib
_collections
collections
types
_functools
functools
contextlib
_virtualenv
_distutils_hack
pywin32_system32
pywin32_bootstrap
importlib._bootstrap
importlib._bootstrap_external
warnings
importlib
importlib._abc
importlib.util
importlib.machinery
snowflake
site
runpy
enum
_sre
sre_constants
sre_parse
sre_compile
_locale
copyreg
re
collections.abc
typing.io
typing.re
typing
ipykernel._version
_json
json.scanner
json.decoder
json.encoder
json
errno
signal
_weakrefset
threading
msvcrt
_winapi
subprocess
jupyter_client._version
platform
zmq.backend.select
cython_runtime
_cython_3_0_0
zmq.error
zmq.backend.cython.context
weakref
zmq.backe

In [71]:
user = os.environ.get("snow_user")
password = os.environ.get("snow_pwd")
account = os.environ.get("snow_acc")
warehouse = config_dict['warehouse']
role = config_dict['role']

database_from_config = config_dict['database']
schema_from_config = config_dict['schema']
view_table_name_from_config = config_dict['view/table name']


connection_parameters = {
    "user": user,
    "password" : password,
    "account": account,
    "role": role,
    "warehouse": warehouse
 }

session = Session.builder.configs(connection_parameters).create()

In [6]:
config_dict['view/table name']

'REPORTS."HR Report"'

In [37]:
session.use_database(database=config_dict['database'])
session.use_schema(schema=config_dict['schema'])
table_ = session.table(config_dict['view/table name'])

In [32]:
session.table(config_dict['view/table name']).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FIRSTNAME"  |"LASTNAME"  |"EMPID"  |"SALARY"  |"POSITION"                |"STATE"  |"ZIP"  |"DOB"       |"SEX"  |"DATEOFHIRE"  |"DATEOFTERMINATION"  |"EMPLOYMENTSTATUS"      |"DEPARTMENT"          |"MANAGERNAME"    |"RECRUITMENTSOURCE"  |"PERFORMANCESCORE"  |"LASTPERFORMANCEREVIEW_DATE"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Wilson       |Adinolfi    |10026    |62506     |Production Technician I   |MA       |1960   |0083-07-10  |M      |2011-0

## TEST CASES START

In [74]:
user = os.environ.get("snow_user")
password = os.environ.get("snow_pwd")
account = os.environ.get("snow_acc")
warehouse = config_dict['warehouse']
role = config_dict['role']

expected_column_count = config_dict["column count"]
database_from_config = config_dict['database']
schema_from_config = config_dict['schema']
view_table_name_from_config = config_dict['view/table name']

folder_name = config_dict["view/table name"].split(".")[0]
file_name = f'''{config_dict['view/table name'].split('.')[1].replace('"','')}.sql'''

In [68]:
def test_count_check():
    return table_.count() > 0

def test_column_validation():
    return len(table_.columns) == expected_column_count and not(len(table_.columns) < expected_column_count)

def test_duplication():
    group_by_ = table_.group_by(table_.columns).count().filter(col('count') > 1).collect()
    return not(bool(group_by_)) == True

def check_pytest():
  exitcode = pytest.main()
  if exitcode == 0:
    query = extract_query(
    database_name=database_from_config,
    schema_name=schema_from_config,
    table_name=view_table_name_from_config,
    session_=session
    )
    if folder_name not in os.listdir(r'./EnterpriseDatabase/'):
        os.mkdir(fr'./EnterpriseDatabase/{folder_name}')

    with open(fr"./EnterpriseDatabase/{folder_name}/{file_name}",'w') as file_:
        file_.write(query)
    

# result = check_pytest()
if __name__ == '__main__':
    check_pytest()

In [56]:
table_.group_by(table_.columns).count().filter(col('count') > 1).collect()

[]

In [69]:
print(test_count_check())
print(test_column_validation())
print(test_duplication())

True
True
False


### TEST CASE - END

In [69]:
df = session.sql('''SELECT GET_DDL('table', 'REPORTS."HR Report"',true)''')


In [27]:
get_ddl = f"SELECT GET_DDL('table','{tb_name}',true)";
df1 = session.sql(get_ddl)

print(df1.collect()[0][0])

create or replace materialized view HR_DB.REPORTS."HR Report"(
	FIRSTNAME,
	LASTNAME,
	EMPID,
	SALARY,
	POSITION,
	STATE,
	ZIP,
	DOB,
	SEX,
	DATEOFHIRE,
	DATEOFTERMINATION,
	EMPLOYMENTSTATUS,
	DEPARTMENT,
	MANAGERNAME,
	RECRUITMENTSOURCE,
	PERFORMANCESCORE,
	LASTPERFORMANCEREVIEW_DATE
) as
select 
SPLIT(TRIM(split(EMPLOYEE_NAME,',')[1]::varchar),' ')[0]::text as FIRSTNAME,TRIM(split(EMPLOYEE_NAME,',')[0]::varchar) as LASTNAME,
-- EMPLOYEE_NAME,
EMPID,SALARY,POSITION,STATE,ZIP,DOB,SEX,DATEOFHIRE,DATEOFTERMINATION,EMPLOYMENTSTATUS,DEPARTMENT,MANAGERNAME,RECRUITMENTSOURCE,PERFORMANCESCORE,LASTPERFORMANCEREVIEW_DATE 
from hr_schema.employee_copy_tbl;


In [71]:
folder_name = config_dict["view/table name"].split(".")[0]
file_name = f'''{config_dict['view/table name'].split('.')[1].replace('"','')}.sql'''

# print(folder_name,file_name)

with open(fr"./EnterpriseDatabase/{folder_name}/{file_name}",'w') as file_:
    file_.write(df.collect()[0][0])

In [75]:
import re

text = 'create or replace view DEV.REP."Diag"'

# pattern = r'(?:\w+\.){1}(.*)'
pattern = r'(.*?)(?:\w+\.){1}(.*)'

match = re.search(pattern, text)

if match:
    print(match.group(2))

REP."Diag"


In [96]:
import re 

# string = 'create or replace view DEV98.REP."Diag"'
string = df.collect()[0][0]

pattern = r'(\w+)\.'


# replaced = re.sub(pattern, '', string)

# print(replaced)

match = re.search(pattern, string)
if match:
#   print(match.group(1) + '.')
  s = f"{match.group(1)}."
  print(string.replace(s,''))

create or replace materialized view REPORTS."HR Report"(
	FIRSTNAME,
	LASTNAME,
	EMPID,
	SALARY,
	POSITION,
	STATE,
	ZIP,
	DOB,
	SEX,
	DATEOFHIRE,
	DATEOFTERMINATION,
	EMPLOYMENTSTATUS,
	DEPARTMENT,
	MANAGERNAME,
	RECRUITMENTSOURCE,
	PERFORMANCESCORE,
	LASTPERFORMANCEREVIEW_DATE
) as
select 
SPLIT(TRIM(split(EMPLOYEE_NAME,',')[1]::varchar),' ')[0]::text as FIRSTNAME,TRIM(split(EMPLOYEE_NAME,',')[0]::varchar) as LASTNAME,
-- EMPLOYEE_NAME,
EMPID,SALARY,POSITION,STATE,ZIP,DOB,SEX,DATEOFHIRE,DATEOFTERMINATION,EMPLOYMENTSTATUS,DEPARTMENT,MANAGERNAME,RECRUITMENTSOURCE,PERFORMANCESCORE,LASTPERFORMANCEREVIEW_DATE 
from hr_schema.employee_copy_tbl;


# ENTITYS

In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.dataframe import col
import os
import json
from query_extractor import extract_query

In [2]:


# Parse the JSON string into a Python dictionary
json_str = open('./config/config.json','r+').read()
config_dict = json.loads(json_str)


user = os.environ.get("snow_user")
password = os.environ.get("snow_pwd")
account = os.environ.get("snow_acc")
warehouse = config_dict['warehouse']
role = config_dict['role']
database_from_config = config_dict['database']
schema_from_config = config_dict['schema']
view_table_name_from_config = config_dict['view/table name']

connection_parameters = {
    "user": user,
    "password" : password,
    "account": account,
    "role": role,
    "warehouse": warehouse
 }

session = Session.builder.configs(connection_parameters).create()








session.use_database(database=config_dict['database'])
session.use_schema(schema=config_dict['schema'])
table = session.table(config_dict['view/table name'])

# df1 = session.sql(f'''SELECT GET_DDL('table','{config_dict["view/table name"]}')''')
# print(df1.collect()[0][0])

In [3]:
query = extract_query(
    database_name=database_from_config,
    schema_name=schema_from_config,
    table_name=view_table_name_from_config,
    session_=session
    )

print(query)

create or replace materialized view REPORTS."HR Report"(
	FIRSTNAME,
	LASTNAME,
	EMPID,
	SALARY,
	POSITION,
	STATE,
	ZIP,
	DOB,
	SEX,
	DATEOFHIRE,
	DATEOFTERMINATION,
	EMPLOYMENTSTATUS,
	DEPARTMENT,
	MANAGERNAME,
	RECRUITMENTSOURCE,
	PERFORMANCESCORE,
	LASTPERFORMANCEREVIEW_DATE
) as
select 
SPLIT(TRIM(split(EMPLOYEE_NAME,',')[1]::varchar),' ')[0]::text as FIRSTNAME,TRIM(split(EMPLOYEE_NAME,',')[0]::varchar) as LASTNAME,
-- EMPLOYEE_NAME,
EMPID,SALARY,POSITION,STATE,ZIP,DOB,SEX,DATEOFHIRE,DATEOFTERMINATION,EMPLOYMENTSTATUS,DEPARTMENT,MANAGERNAME,RECRUITMENTSOURCE,PERFORMANCESCORE,LASTPERFORMANCEREVIEW_DATE 
from hr_schema.employee_copy_tbl;


In [104]:
folder_name = config_dict["view/table name"].split(".")[0]
file_name = f'''{config_dict['view/table name'].split('.')[1].replace('"','')}.sql'''

if folder_name not in os.listdir(r'./EnterpriseDatabase/'):
    os.mkdir(fr'./EnterpriseDatabase/{folder_name}')

In [12]:
import snowflake.snowpark as snp
from snowflake.snowpark import Session
import os
import json

json_str = open('./config/config.json','r+').read()
config_dict = json.loads(json_str)

user = os.environ.get("snow_user")
password = os.environ.get("snow_pwd")
account = os.environ.get("snow_acc")
warehouse = config_dict['warehouse']
role = config_dict['role']


connection_parameters = {
    "user": user,
    "password" : password,
    "account": account,
    "role": role,
    "warehouse": warehouse
 }

session__ = Session.builder.configs(connection_parameters).create()
# session__ = snp.Session()

session__.use_database(database=config_dict['database'])
session__.use_schema(schema=config_dict['schema'])
table_name = session__.table(config_dict['view/table name'])

df = session__.table(table_name)
# ddl = df.describe()

print(df)

TypeError: expected string or bytes-like object